<a href="https://colab.research.google.com/github/chadnanrocksensor/blank-app/blob/main/RUF100_Model_Config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.5 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import streamlit as st
import pandas as pd
import math

# DataFrames (as defined above)
# Code Block 1: Transmitter
transmitter_data = {
    'Code': ['W', 'S', 'D'],
    'Description': ['Wall Mounted', 'Panel Mounted', 'Explosion-proof']
}
transmitter_df = pd.DataFrame(transmitter_data)

# Code Block 2: Transducer
transducer_data = {
    'Code': ['TS-2', 'TM-1', 'TL-1', 'TS-2-HT', 'TM-1-HT', 'TL-1-HT', 'TC-1', 'TC-2'],
    'Description': ['Small (Std. Clamp-on)', 'Medium (Std. Clamp-on)', 'Large (Std. Clamp-on)',
                    'Small (High Temp. Clamp-on)', 'Medium (High Temp. Clamp-on)', 'Large (High Temp. Clamp-on)',
                    'Standard (Insertion)', 'Longer (Insertion)'],
    'Min_Diameter': [15, 50, 300, 15, 50, 300, 80, 80],
    'Max_Diameter': [100, 700, 6000, 100, 700, 6000, 6000, 6000],
    'Min_Temp': [-30, -30, -30, -30, -30, -30, -30, -30],
    'Max_Temp': [90, 90, 90, 160, 160, 160, 160, 160,],
    'Installation_Type': ['Clamp-on', 'Clamp-on', 'Clamp-on', 'Clamp-on', 'Clamp-on', 'Clamp-on','Insertion','Insertion']
}
transducer_df = pd.DataFrame(transducer_data)

# Code Block 4: Pipe Material
pipe_material_data = {
    'Code': ['0', '1', '2', '3', '4', '5', '6'],
    'Description': ['Carbon Steel', 'Stainless Steel', 'Cast Iron', 'FRP', 'PVC', 'Cement', 'Others']
}
pipe_material_df = pd.DataFrame(pipe_material_data)

# Code Block 5: Nominal Pressure
nominal_pressure_data = {
    'Code': ['1', '2', '3', '4'],
    'Description': ['1.0 MPa', '1.6 MPa', '2.5 MPa', '4.0 MPa']
}
nominal_pressure_df = pd.DataFrame(nominal_pressure_data)

# Code Block 7: Temperature Sensor
temperature_sensor_data = {
    'Code': ['N', 'C', 'I', '12', 'S'],
    'Description': ['No Temperature Sensor', 'Clamp-On', 'Insertion', 'Insertion with pressure', 'Small size temp. sensor']
}
temperature_sensor_df = pd.DataFrame(temperature_sensor_data)

# Code Block 8: SD Card Storage
sd_card_data = {
    'Code': ['0', '1'],
    'Description': ['With this function', 'Without this function']
}
sd_card_df = pd.DataFrame(sd_card_data)

def flow_velocity(flow_rate_m3_per_hr, diameter_mm):
    flow_rate_m3_per_sec = flow_rate_m3_per_hr / 3600
    diameter_m = diameter_mm / 1000
    cross_section_area_m2 = math.pi * (diameter_m / 2)**2
    velocity_m_per_s = flow_rate_m3_per_sec / cross_section_area_m2
    return velocity_m_per_s

st.title('RUF100 Model Configurator')

# Input Widgets
transmitter_selection = st.selectbox('Transmitter', transmitter_df['Description'], index=0) # Default to Wall Mounted
pipe_diameter = st.number_input('Pipe Diameter (mm)', min_value=15, max_value=6000, value=15)
pipe_material_selection = st.selectbox('Pipe Material', pipe_material_df['Description'], index=0) # Default to Carbon Steel
nominal_pressure_selection = st.selectbox('Nominal Pressure', nominal_pressure_df['Description'], index=0) # Default to 1.0 MPa
cable_length = st.number_input('Cable Length (meters)', min_value=1, value=5) # Default to 5m
temperature_sensor_selection = st.selectbox('Temperature Sensor', temperature_sensor_df['Description'], index=0) # Default to No Sensor
sd_card_selection = st.selectbox('SD Card Storage', sd_card_df['Description'], index=0) # Default to With SD Card
flow_rate = st.number_input('Flow rate (m³/hr)', min_value=0.0)
temperature_input = st.number_input('Temperature (°C)', min_value=-273.0)

# Transducer Selection Logic
filtered_transducers = transducer_df[
    (transducer_df['Min_Diameter'] <= pipe_diameter) &
    (transducer_df['Max_Diameter'] >= pipe_diameter) &
    (transducer_df['Min_Temp'] <= temperature_input) &
    (transducer_df['Max_Temp'] >= temperature_input)
]

transducer_selection = st.selectbox('Transducer', filtered_transducers['Description'])

# Velocity Check
velocity = flow_velocity(flow_rate, pipe_diameter)
if velocity > 30:
    st.error('Velocity exceeds 30 m/s. Model is not suitable.')

if temperature_input>160:
    st.error('Temperature exceeds 160 degC. Model is not suitable.')

# Model Code Generation
transmitter_code = transmitter_df[transmitter_df['Description'] == transmitter_selection]['Code'].values[0]
transducer_code = filtered_transducers[filtered_transducers['Description'] == transducer_selection]['Code'].values[0]
pipe_material_code = pipe_material_df[pipe_material_df['Description'] == pipe_material_selection]['Code'].values[0]
nominal_pressure_code = nominal_pressure_df[nominal_pressure_df['Description'] == nominal_pressure_selection]['Code'].values[0]
temperature_sensor_code = temperature_sensor_df[temperature_sensor_df['Description'] == temperature_sensor_selection]['Code'].values[0]
sd_card_code = sd_card_df[sd_card_df['Description'] == sd_card_selection]['Code'].values[0]

model_code = f'RUF100-{transmitter_code}-{transducer_code}-{pipe_diameter}-{pipe_material_code}-{nominal_pressure_code}-{cable_length}-{temperature_sensor_code}-{sd_card_code}'
st.write(f'**Model Code:** {model_code}')

Writing app.py


In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.75.58.252:8501

